In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

In [12]:
train_clean = pd.read_csv('train_clean')
test_clean = pd.read_csv('test_clean')
test_copy = pd.read_csv('Test.csv')

In [13]:
!pip3 install lightgbm

In [14]:
print(train_clean.shape)
train_clean

(611772, 297)


,birth_year,Label,ID X PCODE,sex_M,sex_nan,marital_status_M,marital_status_P,marital_status_R,marital_status_S,marital_status_U,marital_status_W,marital_status_f,marital_status_nan,branch_code_30H5,branch_code_49BM,branch_code_748L,branch_code_94KC,branch_code_9F9T,branch_code_BOAS,branch_code_E5SW,branch_code_EU3L,branch_code_O4JC,branch_code_O67J,branch_code_UAOD,branch_code_X23B,...,PCODE_FM3X,PCODE_GHYX,PCODE_GYSR,PCODE_J9JW,PCODE_JWFN,PCODE_JZ9D,PCODE_K6QO,PCODE_LJR9,PCODE_N2MW,PCODE_P5DA,PCODE_PYUQ,PCODE_QBOL,PCODE_RIBP,PCODE_RVSZ,PCODE_SOP4,PCODE_nan,join_day,join_month,join_year,year_diff,birth_year_freq_encode,join_day_freq_encode,join_month_freq_encode,join_year_freq_encode,year_diff_freq_encode
0,1987,0,4WKQSBB X P5DA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,2.0,2019.0,32.0,22155,611520.0,40782.0,186984.0,22176.0
1,1981,0,CP5S02H X P5DA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,6.0,2019.0,38.0,21147,611520.0,53193.0,186984.0,22407.0
2,1991,0,2YKDILJ X P5DA,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,6.0,2013.0,22.0,14511,611520.0,53193.0,12831.0,6930.0
3,1990,0,2S9E81J X P5DA,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,8.0,2019.0,29.0,16842,611520.0,45717.0,186984.0,20244.0
4,1990,0,BHDYVFT X P5DA,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,8.0,2019.0,29.0,16842,611520.0,45717.0,186984.0,20244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,1974,0,8SBA93U X ECY3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,9.0,2019.0,45.0,20202,611520.0,43533.0,186984.0,18648.0
611768,1987,0,Y73ETTO X ECY3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,11.0,2018.0,31.0,22155,611520.0,43617.0,154392.0,21903.0
611769,1985,0,MVXG2LC X ECY3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,2019.0,34.0,22197,611520.0,40782.0,186984.0,24087.0
611770,1989,0,BUQ8DQN X ECY3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,7.0,2019.0,30.0,20916,611520.0,43743.0,186984.0,21819.0


In [15]:
y = train_clean.Label
X = train_clean.drop(['Label', 'ID X PCODE'], axis=1)

In [ ]:
# train validation split
import random
random.seed(1234)
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state =42)
print('X_train: ', X_train.shape, 'X_test: ', X_test.shape, 'y_train: ', y_train.shape, 'y_test: ', y_test.shape)

##  CatBoost Classifier

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=200, learning_rate=0.01, l2_leaf_reg=3.5, depth=8, rsm=0.98, loss_function= 'Logloss', 
                       eval_metric='AUC', random_seed=42)
cat.fit(X_train, y_train)
#
# Predicting the Test set results
y_pred = cat.predict(X_test)
# metric accuracy
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))
#
from sklearn.metrics import log_loss
print('Log Loss: ', log_loss(y_test, cat.predict_proba(X_test)))

## LightGBM Classifier

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
#
y_pred_clf = clf.predict(X_test)
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_clf, normalize=True, sample_weight=None))
#
from sklearn.metrics import log_loss
print('Log Loss: ', log_loss(y_test, clf.predict_proba(X_test)))

### Submission

In [ ]:
true_values = []
for v in test_clean.values:
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in test_clean.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

In [ ]:
samplesub = pd.read_csv('SampleSubmission.csv')
samplesub.head()

In [ ]:
proba = clf.predict_proba(test_clean.drop(['ID X PCODE'], axis=1))[:, 1]
#
sub = test_clean[['ID X PCODE']] #return the column as a new dataframe
sub['Label'] = proba

In [ ]:
sub.columns = ['ID X PCODE', 'Label']
for i in range(sub.shape[0]):
    if sub['ID X PCODE'].iloc[i] in true_values:
        sub['Label'].iloc[i] = 1.0

sub.reset_index(drop=True, inplace=True)
sub.to_csv('submission3.csv', index=False)